## 7.3 Multidimensional integrals

Explain the ways how to use or generalise the one-dimensional approaches for computing multidi-mensional integrals. The script integrals_2D.m implements some possibilities. Explain the used procedures.

(*) There are various implementation of the same methods. You can profile the code and explain some weak performance issues of Matlab (Python in our case) according to the results

In [26]:
import numpy as np
from typing import Callable
import time

In [27]:
# Example function to integrate
def f(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    return np.abs(np.sin(np.pi ** 2 * ((x - 0.5) ** 2 + (y - 0.5) ** 2)))

f(0.5, 0.5)  # Example call to the function

np.float64(0.0)

In [28]:
def compute_2d_integral(
    f: Callable[[np.ndarray, np.ndarray], np.ndarray],
    xlim: tuple[float, float],
    ylim: tuple[float, float],
    h: float,
    method: str
) -> tuple[float, float]:
    xmin, xmax = xlim
    ymin, ymax = ylim

    if method == '2D_midpoint_matrix2':
        xgrid, ygrid = np.meshgrid(np.arange(xmin + h / 2, xmax, h),
                                   np.arange(ymin + h / 2, ymax, h))
        tic = time.time()
        fvals = f(xgrid, ygrid)
        integral = np.sum(fvals) * h ** 2
        toc = time.time()

    elif method == '2D_midpoint_loop2':
        xgrid = np.arange(xmin + h / 2, xmax, h)
        ygrid = np.arange(ymin + h / 2, ymax, h)
        tic = time.time()
        integral = 0.0
        for i in range(len(xgrid)):
            for j in range(len(ygrid)):
                integral += f(xgrid[i], ygrid[j])
        integral *= h ** 2
        toc = time.time()

    elif method == '1D_x+y_rectangle_loop3':
        xgrid = np.arange(xmin, xmax + h, h)
        ygrid = np.arange(ymin, ymax + h, h)
        tic = time.time()
        integral = 0.0
        firstrun = True
        for k1 in range(len(xgrid)):
            integral_y2 = 0.5 * f(xgrid[k1], ygrid[0])
            for k2 in range(1, len(ygrid) - 1):
                integral_y2 += f(xgrid[k1], ygrid[k2])
            integral_y2 += 0.5 * f(xgrid[k1], ygrid[-1])
            if not firstrun:
                integral += 0.5 * (integral_y1 + integral_y2)
            integral_y1 = integral_y2
            firstrun = False
        integral *= h ** 2
        toc = time.time()

    else:
        raise ValueError("Unknown method")

    return integral, toc - tic

In [29]:
exact = 0.6551724745288948
h = 0.01
xmin = 0.0
xmax = 1.0
ymin = 0.0
ymax = 1.0

methods = [
    '2D_midpoint_matrix2',
    '2D_midpoint_loop2',
    '1D_x+y_rectangle_loop3'
]
print(f"Exact integral: {exact:.10f}\n")

# Loop over methods
for method in methods:
    print(f"Method: {method}")
    result, duration = compute_2d_integral(f, [xmin, xmax], [ymin, ymax], h, method)
    precision = abs(result - exact) / abs(exact)
    print(f"Time: {duration:.4f} s; Precision: {precision:.6e}; Value: {result:.10f}\n")

Exact integral: 0.6551724745

Method: 2D_midpoint_matrix2
Time: 0.0007 s; Precision: 6.276570e-05; Value: 0.6552135969

Method: 2D_midpoint_loop2
Time: 0.0128 s; Precision: 6.276570e-05; Value: 0.6552135969

Method: 1D_x+y_rectangle_loop3
Time: 0.0123 s; Precision: 7.956738e-05; Value: 0.6551203442



# 2D Numerical Integration Results – Explanation

We evaluated a 2D integral over the unit square `[0,1] × [0,1]` for the function above.

We tested three numerical integration methods using a step size `h = 0.01`:

1. `2D_midpoint_matrix2`

- **Type**: Midpoint rule (vectorized)
- **Description**: Uses midpoint sampling and full matrix evaluation with `meshgrid`.
- **Performance**: Fastest due to full vectorization.
- **Time**: ~0.0006 seconds
- **Precision**: ~6.28e-5 (very accurate)

**Use case**: Best for performance when memory usage is acceptable.

2. `2D_midpoint_loop2`

- **Type**: Midpoint rule (with nested loops)
- **Description**: Loops through midpoint samples manually.
- **Performance**: Much slower due to explicit iteration.
- **Time**: ~0.0447 seconds
- **Precision**: Same as vectorized version (as expected)

**Use case**: Educational or low-memory environments; otherwise inefficient.
**Note**: After my experiments, I found out that using h smaller than 0.00001, the matrix is taking too much memory and may cause an overflow.


3. `1D_x+y_rectangle_loop3`

- **Type**: Rectangle rule (optimized loop version)
- **Description**: Applies 1D rectangle rule in `y` then integrates across `x`, minimizing redundant calls.
- **Performance**: Slower than vectorized midpoint but faster than naive loops.
- **Time**: ~0.0414 seconds
- **Precision**: ~7.96e-5 (slightly less accurate than midpoint)

**Use case**: When midpoint rule isn't available or rectangle logic is preferred.